In [1]:
import re

In [2]:
def align(input_file, output_file, delta_x, delta_y):
    """
    读取.gcode文件，将所有X后面的数字减1，然后保存到新文件。

    Args:
        input_file (str): 输入.gcode文件的路径。
        output_file (str): 输出.gcode文件的路径。
    """

    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            # 使用更精确的正则表达式查找并修改X坐标
            line = re.sub(r'([XY])(\s*-?\d*\.?\d+)', lambda m: f'{m.group(1)}{float(m.group(2)) - delta_x:.3f}' if m.group(1) == 'X' else f'{m.group(1)}{float(m.group(2)) - delta_y:.3f}', line)
            f_out.write(line)

In [3]:
def add_M7_M9(input_file, output_file):
    """
    读取.gcode文件，在所有以G0开头的行前面加一行“M7”，后面加一行“M9”，然后保存到新文件。

    Args:
        input_file (str): 输入.gcode文件的路径。
        output_file (str): 输出.gcode文件的路径。
    """

    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            # 在以G0开头的行前面加一行“M7”，后面加一行“M9”
            if line.startswith("G0"):
                f_out.write("M9\n")
                f_out.write(line)
                f_out.write("M7\n")
            else:
                f_out.write(line)

In [4]:
def modify_head(input_file, output_file):
    """
    读取.gcode文件，修改头部信息，然后保存到新文件。

    Args:
        input_file (str): 输入.gcode文件的路径。
        output_file (str): 输出.gcode文件的路径。
    """

    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        # 读取头部信息
        header_lines = []
        
        print(f_in)
        
        count = 0
        for line in f_in:
            if count < 28:
                header_lines.append(line)
                count += 1
                if count == 28:
                    break

        # 修改头部信息
        header_lines = [
            f"{header_lines[0]}",  # FLAVOR
            f"{header_lines[1]}",  # TIME
            f"{header_lines[2]}",  # Filament used
            f"{header_lines[3]}",  # Layer height
            f"{header_lines[24]}", # LAYER_COUNT
            f"{header_lines[25]}", # LAYER:0
            f"{header_lines[27]}", # MESH:filename
            "\nG21 ;start of the code",
            "\nG1 Z15 F300",
            "\nG28 X0 Y0 ;Home",
            "\nG92 X0 Y0 ;Consider this as current",
            "\nG0 X50 Y50 F3000 ;Go-to Offset",  
            "\nG92 X0 Y0 ;Reset",
            "\n\n"
        ]

        # 写入新的头部信息
        f_out.writelines(header_lines)

        # 处理剩余的 G 代码行
        for line in f_in:
            f_out.write(line)

In [7]:
# 设置输入和输出文件路径
input_file = "7.22//side_20_20.gcode"
output_file = "tmp3.gcode"

# 调用函数修改G代码文件
align(input_file, output_file, -25, -25)

: 

In [10]:
# 设置输入和输出文件路径
input_file = "tmp\platform1315_zigzag_0.4_40.gcode"
tmp_file = "tmp.gcode"
output_file = "7.15\platform1315_zigzag_0.4_40.gcode"

# 调用函数修改G代码文件
align(input_file, tmp_file, 124.662, 115.054)
modify_head(tmp_file, output_file)

<_io.TextIOWrapper name='tmp.gcode' mode='r' encoding='UTF-8'>
